# Overview

In this tutorial we will explore how to create a simple convolutional neural network (CNN) for classification of brain tumors from MRI.

## Workshop Links

Use the following link to access materials from this workshop: https://github.com/peterchang77/dl_tutor/tree/master/workshops

*Tutorials*

* Introduction to Tensorflow 2.0 and Keras: https://bit.ly/2VSYaop
* CNN for pneumonia classification: https://bit.ly/2D9ZBrX
* CNN for pneumonia segmentation: https://bit.ly/2VQMWk9
* CNN for brain tumor classification: https://bit.ly/44eekdt (**current tutorial**)
* CNN for brain tumor segmentation: https://bit.ly/449i5Ro 

# Environment

The following lines of code will configure your Google Colab environment for this tutorial.

### Enable GPU runtime

Use the following instructions to switch the default Colab instance into a GPU-enabled runtime:

```
Runtime > Change runtime type > Hardware accelerator > GPU
```

### Jarvis library

In this notebook we will Jarvis, a custom Python package to facilitate data science and deep learning for healthcare. Among other things, this library will be used for low-level data management, stratification and visualization of high-dimensional medical data.

In [ ]:
# --- Install Jarvis library
%pip install jarvis-md

### Imports

Use the following lines to import any needed libraries:

In [ ]:
import numpy as np, pandas as pd
from tensorflow import losses, optimizers
from tensorflow.keras import Input, Model, models, layers, metrics
from jarvis.train import datasets
from jarvis.utils.display import imshow

# Data

The data used in this tutorial will consist of brain tumor MRI exams derived from the MICCAI Brain Tumor Segmentation Challenge (BRaTS). More information about he BRaTS Challenge can be found here: http://braintumorsegmentation.org/. Each single 2D slice will consist of one of four different sequences (T2, FLAIR, T1 pre-contrast and T1 post-contrast). In this exercise, we will use this dataset to derive a model for slice-by-slice tumor detection. The custom `datasets.download(...)` method can be used to download a local copy of the dataset. By default the dataset will be archived at `/data/raw/mr_brats_2020`; as needed an alternate location may be specified using `datasets.download(name=..., path=...)`. 

In [ ]:
# --- Download dataset
datasets.download(name='mr/brats-2020-mip')

Once downloaded, the `datasets.prepare(...)` method can be used to generate the required python Generators to iterate through the dataset, as well as a `client` object for any needed advanced functionality. As needed, pass any custom configurations (e.g. batch size, normalization parameters, etc) into the optional `configs` dictionary argument. 

Additionally, we will re-use this dataset for different tasks (e.g., tumor detection, survival prediction, etc). To specificy the correct Generator template file, pass a designated `keyword` string. In this tutorial, we will be using brain MRI volumes that have been preprocessed using a *mean intensity projection* (MIP) algorithm to subsample the original 155-slice inputs to 40-50 slices, facilitating ease of algorithm training within the Google Colab platform. In addition we will be performing global (entire-slice) tumor prediction. To select the correct Client template for this task, use the keyword string `mip*glb`. 

In [ ]:
# --- Prepare generators
gen_train, gen_valid, client = datasets.prepare(name='mr/brats-2020-mip', keyword='mip*glb')

As before, each iteration yields two variables, `xs` and `ys`, each representing a dictionary of model input(s) and output(s). In the current example, there is just a single input and output. Let us examine the generator data:

In [ ]:
# --- Yield one example
xs, ys = next(gen_train)

# --- Print dict keys
print('xs keys: {}'.format(xs.keys()))
print('ys keys: {}'.format(ys.keys()))

In [ ]:
# --- Print data shape
print('xs shape: {}'.format(xs['dat'].shape))
print('ys shape: {}'.format(ys['tumor'].shape))

### BRATS Data

Let us get to know this data a little bit more. The input images in the variable `dat` are matrices of shape `1 x 240 x 240 x 4`. Note that even though the images here are 2D in shape, the full matrix is a 3D tensor `(z, y, x)` where `z = 1` in this implementation. Note that although the 3rd z-axis dimension is redundant here (for a single slice input), many of our more complex models and architectures will commonly require a full 3D tensor. Because of this, we will directly use 3D convolutions throughout the tutorial materials for consistency.

In [ ]:
print(xs['dat'][0].shape)

The four channels represent four different input MRI sequences. Each sequence is used to evaluate for a different tissue quality. T2 and FLAIR are used to evaluate edema (fluid) that results from brain injury. T1 images are used to evaluate anatomy and breakdown of the blood-brain-barrier through contrast leakge.  

```
dat[..., 0] = FLAIR
dat[..., 1] = T1 precontrast
dat[..., 2] = T1 postcontrast
dat[..., 3] = T2
```

To visualize these different modalities run the following cell:

In [ ]:
imshow(xs['dat'][..., 0].astype('float32'), title='FLAIR')
imshow(xs['dat'][..., 1].astype('float32'), title='T1 precontrast')
imshow(xs['dat'][..., 2].astype('float32'), title='T1 postcontrast')
imshow(xs['dat'][..., 3].astype('float32'), title='T2')

### Model inputs

For every input in `xs`, a corresponding `Input(...)` variable should be created for model development:

In [ ]:
# --- Create model inputs
inputs = {}
inputs['dat'] = Input(shape=(1, 240, 240, 4))

# Convolutional Operations

In this tutorial, a CNN will be created using 3D convolutional operations. A convolutional operation is defined by the following minimum specifications:

* filter / channel depth
* kernel size
* strides
* padding

For a good review of convolutional math, see: http://deeplearning.net/software/theano/tutorial/conv_arithmetic.html

To instantiate a convolutional layer in Keras:

In [ ]:
# --- Define regular convolution
l1 = layers.Conv3D(
    filters=16, 
    kernel_size=(1, 3, 3), 
    strides=(1, 1, 1), 
    padding='same')(inputs['dat'])

# --- Define strided convolution
l1 = layers.Conv3D(
    filters=16, 
    kernel_size=(1, 3, 3), 
    strides=(1, 2, 2), 
    padding='same')(inputs['dat'])

To reuse identical function arguments, consider maintaining a `kwargs` dictionary and pass using the `**` symbol:

In [ ]:
# --- Define kwargs dictionary
kwargs = {
    'kernel_size': (1, 3, 3),
    'padding': 'same'}

# ---- Define stack of convolutions
l1 = layers.Conv3D(filters=16, strides=(1, 1, 1), **kwargs)(inputs['dat'])
l2 = layers.Conv3D(filters=32, strides=(1, 1, 1), **kwargs)(l1)

## Blocks

In addition to the requisite convolutional operationss and activation functions, batch normalization is almost universally used in modern CNN architectures. Thus at minimum, a common baseline *block* pattern of operations can be defined as:

* convolutional operation
* batch normalization
* activation function (e.g. ReLU)

Let us define a block:

In [ ]:
# --- Define block
c1 = layers.Conv3D(filters=16, **kwargs)(inputs['dat'])
n1 = layers.BatchNormalization()(c1)
r1 = layers.ReLU()(n1)

During the course of buildling CNNs, we will be writing **many** blocks over time. Thus for brevity, let us use lambda functions to define modular, reusable components:

In [ ]:
# --- Define lambda functions
conv = lambda x, filters, strides : layers.Conv3D(filters=filters, strides=strides, **kwargs)(x)
norm = lambda x : layers.BatchNormalization()(x)
relu = lambda x : layers.ReLU()(x)

Great! Now let us rewrite a block using lambda shorthand:

In [ ]:
# --- Define block
b1 = relu(norm(conv(inputs['dat'], 16, (1, 1, 1))))

In practice, the two most common **block patterns** will be regular convolutional block and a strided convolutional block (for subsampling). Let us then create two more high-level lambda functions for this:

In [ ]:
# --- Define stride-1, stride-2 blocks
conv1 = lambda filters, x : relu(norm(conv(x, filters, strides=1)))
conv2 = lambda filters, x : relu(norm(conv(x, filters, strides=(1, 2, 2))))

Now let us see how easy it is to create series of alternating stride-1 and stride-2 blocks:

In [ ]:
# --- Define series of blocks
l1 = conv1(16, inputs['dat'])
l2 = conv1(24, conv2(24, l1))
l3 = conv1(32, conv2(32, l2))
l4 = conv1(48, conv2(48, l3))
l5 = conv1(64, conv2(64, l4))
l6 = conv1(80, conv2(80, l5))
l7 = conv1(96, conv2(96, l6))

# Fully Connected Layers

After a series of convolutional block operations, a CNN must transition to MLP type operations (e.g. matrix multiplications). To convert 3D (or 4D) feature maps into vectors, consider one of the following approaches:

* serial convolutions (with stride > 1 or VALID type padding)
* global pool operations (mean or max)
* reshape / flatten operation

Of these, the reshape / flatten operation is perhaps the most simple and easiest to use, and requires no additional learnable parameters. The following line of code implements the reshape / flatten operation:

In [ ]:
f0 = layers.Reshape([1, 1, 1, -1])(l7)

### MLP

Now that the intermediate layer is defined as a vector, a number of standard MLP type operations may be performed, including creation of an arbitrary (optional) number of hidden layers. Once ready, the final layer should be implemented using a standard matrix multiplication operation (`layers.Dense(...)` object) that yields a vector of logit scores **without** any activation function applied.

In [ ]:
# --- Final logit scores
logits = {}
logits['tumor'] = layers.Conv3D(filters=2, kernel_size=1, name='tumor')(f0)

# Model

Putting everything together, use the following cell to create and compile the convolutional neural network:

In [ ]:
# --- Create model
model = Model(inputs=inputs, outputs=logits)

# --- Compile model
model.compile(
    optimizer=optimizers.Adam(learning_rate=2e-4), 
    loss={'tumor': losses.SparseCategoricalCrossentropy(from_logits=True)}, 
    metrics={'tumor': 'sparse_categorical_accuracy'})

# Model Training

### In-Memory Data

The following line of code will load all training data into RAM memory. This strategy can be effective for increasing speed of training for small to medium-sized datasets.

In [ ]:
# --- Load data into memory
client.load_data_in_memory()

### Training

Once the model has been compiled and the data prepared (via a generator), training can be invoked using the `model.fit(...)` method. Ensure that both the training and validation data generators are used. In this particular example, we are defining arbitrary epochs of 100 steps each. Training will proceed for 8 epochs in total. Validation statistics will be assess every fourth epoch. As needed, tune these arugments as need.

In [ ]:
model.fit(
    x=gen_train, 
    steps_per_epoch=100, 
    epochs=8,
    validation_data=gen_valid,
    validation_steps=100,
    validation_freq=4)

# Evaluation

To test the trained model, the following steps are required:

* load data
* use `model.predict(...)` to obtain logit scores
* use `np.argmax(...)` to obtain prediction
* compare prediction with ground-truth

Recall that the generator used to train the model simply iterates through the dataset randomly. For model evaluation, the cohort must instead be loaded manually in an orderly way. For this tutorial, we will create new **test mode** data generators, which will simply load each example individually once for testing. 

In [ ]:
# --- Create validation generator
test_train, test_valid = client.create_generators(test=True)

Use the following lines of code to loop through the test set generator and run model prediction on each example:

In [ ]:
# --- Test model
pred = []
true = []

for x, y in test_valid:
    
    x['dat'] = x['dat'].reshape(-1, 1, 240, 240, 4)
    logits = model.predict(x)
    
    if type(logits) is dict:
        logits = logits['tumor']
        
    argmax = np.argmax(logits[0], axis=-1)
    
    pred.append(np.squeeze(argmax))
    true.append(np.squeeze(y['tumor']))

Use the following lines of code to calculate validataion cohort performance:

In [ ]:
# --- Calculate accuracy
pred = np.array(pred)
true = np.array(true)

correct = np.sum(pred == true)

print('Accuracy: {:0.5f}'.format(correct / pred.size))

## Saving and Loading a Model

After a model has been successfully trained, it can be saved and/or loaded by simply using the `model.save()` and `models.load_model()` methods. 

In [ ]:
# --- Serialize a model
model.save('./cnn.hdf5')

In [ ]:
# --- Load a serialized model
del model
model = models.load_model('./cnn.hdf5', compile=False)